In [62]:
from statsbombpy import sb
import pandas as pd
import json
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
competitions = sb.competitions()

/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [4]:
competitions

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
1,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-02-14T05:41:27.566989,None,None,2024-02-14T05:41:27.566989
2,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
3,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
4,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
66,55,43,Europe,UEFA Euro,male,False,True,2020,2023-02-24T21:26:47.128979,2023-04-27T22:38:34.970148,2023-04-27T22:38:34.970148,2023-02-24T21:26:47.128979
67,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2023-06-18T19:28:39.443883,2021-06-13T16:17:31.694,None,2023-06-18T19:28:39.443883
68,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2023-10-24T03:36:54.066267,2023-10-24T03:37:29.085948,2023-10-24T03:37:29.085948,2023-10-24T03:36:54.066267
69,72,107,International,Women's World Cup,female,False,True,2023,2023-12-12T14:06:50.626363,2023-12-12T14:12:41.561162,2023-12-12T14:12:41.561162,2023-12-12T14:06:50.626363


In [5]:
competitions['competition_name'].unique()

array(['1. Bundesliga', 'African Cup of Nations', 'Champions League',
       'Copa del Rey', "FA Women's Super League", 'FIFA U20 World Cup',
       'FIFA World Cup', 'Indian Super league', 'La Liga',
       'Liga Profesional', 'Ligue 1', 'Major League Soccer',
       'North American League', 'NWSL', 'Premier League', 'Serie A',
       'UEFA Euro', 'UEFA Europa League', "UEFA Women's Euro",
       "Women's World Cup"], dtype=object)

In [6]:
competitions[
    (competitions['competition_name'].isin([
    'Premier League', '1. Bundesliga', 'La Liga', 'Ligue 1', 'Serie A']))
             & 
    (competitions['season_name']=='2015/2016')
            ]

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
41,11,27,Spain,La Liga,male,False,False,2015/2016,2023-09-20T17:21:32.111535,2021-06-13T16:17:31.694,None,2023-09-20T17:21:32.111535
58,7,27,France,Ligue 1,male,False,False,2015/2016,2023-12-13T00:27:57.162379,None,None,2023-12-13T00:27:57.162379
62,2,27,England,Premier League,male,False,False,2015/2016,2023-11-14T11:25:33.476498,2021-06-13T16:17:31.694,None,2023-11-14T11:25:33.476498
64,12,27,Italy,Serie A,male,False,False,2015/2016,2023-12-13T17:32:46.423081,None,None,2023-12-13T17:32:46.423081


In [7]:
#competitions[competitions['competition_name']=='Champions League']

In [55]:
PremierLeague = sb.matches(competition_id=2, season_id=27)
PremierLeague

/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3754058,2016-01-02,16:00:00.000,England - Premier League,2015/2016,Leicester City,AFC Bournemouth,0,0,available,...,2021-06-12T16:17:31.694,20,Regular Season,King Power Stadium,Andre Marriner,Claudio Ranieri,Eddie Howe,1.1.0,2,2
1,3754245,2015-10-17,16:00:00.000,England - Premier League,2015/2016,West Bromwich Albion,Sunderland,1,0,available,...,2021-06-13T16:17:31.694,9,Regular Season,The Hawthorns,Martin Atkinson,Tony Pulis,Sam Allardyce,1.1.0,2,2
2,3754136,2015-12-19,18:30:00.000,England - Premier League,2015/2016,Newcastle United,Aston Villa,1,1,available,...,2021-06-13T16:17:31.694,17,Regular Season,St. James'' Park,Martin Atkinson,Steve McClaren,Rémi Garde,1.1.0,2,2
3,3754037,2016-04-30,16:00:00.000,England - Premier League,2015/2016,Everton,AFC Bournemouth,2,1,available,...,2021-06-12T16:17:31.694,36,Regular Season,Goodison Park,Neil Swarbrick,Roberto Martínez Montoliú,Eddie Howe,1.1.0,2,2
4,3754039,2016-02-13,16:00:00.000,England - Premier League,2015/2016,Crystal Palace,Watford,1,2,available,...,2021-06-12T16:17:31.694,26,Regular Season,Selhurst Park,Robert Madley,Alan Pardew,Enrique Sánchez Flores,1.1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,3754020,2015-08-17,21:00:00.000,England - Premier League,2015/2016,Liverpool,AFC Bournemouth,1,0,available,...,2021-06-13T16:17:31.694,2,Regular Season,Anfield,Craig Pawson,Brendan Rodgers,Eddie Howe,1.1.0,2,2
376,3754267,2015-08-15,16:00:00.000,England - Premier League,2015/2016,Watford,West Bromwich Albion,0,0,available,...,2021-06-13T16:17:31.694,2,Regular Season,Vicarage Road,Paul Tierney,Enrique Sánchez Flores,Tony Pulis,1.1.0,2,2
377,3754141,2015-08-09,14:30:00.000,England - Premier League,2015/2016,Arsenal,West Ham United,0,2,available,...,2021-06-13T16:17:31.694,1,Regular Season,Emirates Stadium,Martin Atkinson,Arsène Wenger,Slaven Bilić,1.1.0,2,2
378,3754128,2015-08-08,16:00:00.000,England - Premier League,2015/2016,AFC Bournemouth,Aston Villa,0,1,available,...,2021-06-13T16:17:31.694,1,Regular Season,Vitality Stadium,Mark Clattenburg,Eddie Howe,Tim Sherwood,1.1.0,2,2


In [61]:
relevant_attributes = ['minute', 'team', 'shot_statsbomb_xg','shot_outcome', 'location', 'shot_aerial_won', 'shot_body_part', 'shot_deflected',
       'shot_end_location', 'shot_first_time', 'shot_one_on_one', 
       'shot_saved_off_target',  'shot_technique',
       'shot_type', 'type']

league_df = pd.DataFrame(columns=relevant_attributes)

for match in PremierLeague['match_id']:
    match_df = sb.events(match_id=match)#[relevant_attributes]#.dropna(subset=['shot_statsbomb_xg'])
    league_df = pd.concat([league_df, match_df], ignore_index=True)
league_df
#
#print(match_df)

/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/var/folders/nw/96mf1fcn3bq42czp2yzy6sdr0000gn/T/ipykernel_27171/3140890475.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  league_df = pd.concat([league_df, match_df], ignore_index=True)
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/en

/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credenti

KeyboardInterrupt: 

In [9]:
events = sb.events(match_id=3754078)

/Users/Senyo/miniconda3/envs/dsc/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [10]:
events

,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,...,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Chelsea,33,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Swansea City,26,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Swansea City,26,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Chelsea,33,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Chelsea,33,00:00:00.000,Half Start,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Swansea City,26,00:48:00.495,Half End,NaN
3348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Chelsea,33,00:49:31.473,Half End,NaN
3349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Swansea City,26,00:49:31.473,Half End,NaN
3350,Yellow Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Chelsea,33,00:08:03.366,Bad Behaviour,NaN


In [48]:
events.columns

Index(['bad_behaviour_card', 'ball_receipt_outcome', 'ball_recovery_offensive',
       'ball_recovery_recovery_failure', 'block_deflection', 'block_offensive',
       'block_save_block', 'carry_end_location', 'clearance_aerial_won',
       'clearance_body_part', 'clearance_head', 'clearance_left_foot',
       'clearance_right_foot', 'counterpress', 'dribble_nutmeg',
       'dribble_outcome', 'dribble_overrun', 'duel_outcome', 'duel_type',
       'duration', 'foul_committed_advantage', 'foul_committed_card',
       'foul_committed_penalty', 'foul_committed_type', 'foul_won_advantage',
       'foul_won_defensive', 'foul_won_penalty', 'goalkeeper_body_part',
       'goalkeeper_end_location', 'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_punched_out', 'goalkeeper_shot_saved_off_target',
       'goalkeeper_technique', 'goalkeeper_type', 'id', 'index',
       'injury_stoppage_in_chain', 'interception_outcome', 'location',
       'match_id', 'minute', 'miscontrol_aerial_won'

In [31]:
relevant_attributes = ['minute', 'team', 'shot_statsbomb_xg','shot_outcome', 'location', 'shot_aerial_won', 'shot_body_part', 'shot_deflected',
       'shot_end_location', 'shot_first_time', 'shot_one_on_one', 
       'shot_saved_off_target',  'shot_technique',
       'shot_type', 'type']

In [41]:
relevant = events[relevant_attributes]
events[events['type']=='Own Goal Against'].dropna(axis=1)

,duration,id,index,location,match_id,minute,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,team,team_id,timestamp,type
3339,0.0,c49d8042-3ef1-4a10-acf9-1e0bc3eec027,1112,"[7.2, 53.2]",3754078,29,1,From Kick Off,Federico Fernández,3290.0,Right Center Back,66,Chelsea,33,[32124451-ae8e-40b0-82fa-a450314b4e0c],49,Swansea City,26,00:29:49.682,Own Goal Against


In [32]:
chelsea_swansea = events[relevant_attributes].dropna(subset=['shot_statsbomb_xg'])

In [15]:
base_url = 'https://understat.com/match/'
match = str(input('Please enter match id: '))
url = base_url + match

Please enter match id: 86


In [16]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')
#help(BeautifulSoup)

In [17]:
#scripts

In [18]:
strings = scripts[1].string

In [19]:
#strings

In [20]:
ind_start = strings.index("('")+2
ind_end = strings.index("')")

json_data = strings[ind_start:ind_end]
json_data = json_data.encode('utf8').decode('unicode_escape')

data = json.loads(json_data)


In [21]:
data

{'h': [{'id': '51525',
   'minute': '10',
   'result': 'MissedShots',
   'X': '0.7609999847412109',
   'Y': '0.225',
   'xG': '0.01549530029296875',
   'player': 'Branislav Ivanovic',
   'h_a': 'h',
   'player_id': '682',
   'situation': 'OpenPlay',
   'season': '2015',
   'shotType': 'RightFoot',
   'match_id': '86',
   'h_team': 'Chelsea',
   'a_team': 'Swansea',
   'h_goals': '2',
   'a_goals': '2',
   'date': '2015-08-08 20:30:00',
   'player_assisted': 'Willian',
   'lastAction': 'Pass'},
  {'id': '51528',
   'minute': '17',
   'result': 'BlockedShot',
   'X': '0.7269999694824218',
   'Y': '0.34599998474121096',
   'xG': '0.014681600034236908',
   'player': 'Nemanja Matic',
   'h_a': 'h',
   'player_id': '697',
   'situation': 'OpenPlay',
   'season': '2015',
   'shotType': 'LeftFoot',
   'match_id': '86',
   'h_team': 'Chelsea',
   'a_team': 'Swansea',
   'h_goals': '2',
   'a_goals': '2',
   'date': '2015-08-08 20:30:00',
   'player_assisted': 'Eden Hazard',
   'lastAction': 'Pa

In [22]:
x = []
y = []
minute = []
xG = []
h_a = []
team = []
result = []

data_home = data['h']
data_away = data['a']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'h_a':
            h_a.append(data_home[index][key])
        if key == 'minute':
            minute.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'h_a':
            h_a.append(data_away[index][key])
        if key == 'minute':
            minute.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [23]:
col_names = ['minute','team', 'x', 'y', 'xG','h_a', 'result']
df = pd.DataFrame([minute, team, x,y,xG, h_a, result], index=col_names)
df = df.T

In [24]:
for col in ['x', 'y', 'xG', 'minute']:
    df[col]=pd.to_numeric(df[col])
df.round(decimals=5)
df.sort_values('minute')

,minute,team,x,y,xG,h_a,result
11,6,Swansea,0.917,0.596,0.038043,a,MissedShots
0,10,Chelsea,0.761,0.225,0.015495,h,MissedShots
13,14,Swansea,0.812,0.363,0.035972,a,SavedShot
12,14,Swansea,0.885,0.441,0.365917,a,BlockedShot
1,17,Chelsea,0.727,0.346,0.014682,h,BlockedShot
2,22,Chelsea,0.838,0.845,0.040702,h,Goal
16,28,Swansea,0.929,0.537,0.510695,a,Goal
15,28,Swansea,0.937,0.524,0.556185,a,BlockedShot
14,28,Swansea,0.924,0.451,0.312095,a,SavedShot
17,29,Swansea,0.057,0.295,0.000000,a,OwnGoal


In [33]:
chelsea_swansea

,minute,team,shot_statsbomb_xg,shot_outcome,location,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_one_on_one,shot_saved_off_target,shot_technique,shot_type,type
3267,6,Swansea City,0.071600,Off T,"[114.7, 31.9]",True,Head,NaN,"[120.0, 46.2, 0.8]",NaN,NaN,NaN,Normal,Open Play,Shot
3268,10,Chelsea,0.004586,Off T,"[89.6, 66.2]",NaN,Right Foot,NaN,"[120.0, 49.0, 7.4]",NaN,NaN,NaN,Normal,Open Play,Shot
3269,14,Swansea City,0.181297,Blocked,"[106.0, 44.8]",NaN,Left Foot,NaN,"[107.0, 44.2]",NaN,NaN,NaN,Normal,Open Play,Shot
3270,14,Swansea City,0.045967,Saved,"[101.6, 51.7]",NaN,Left Foot,NaN,"[115.5, 42.5, 1.3]",NaN,NaN,NaN,Half Volley,Open Play,Shot
3271,17,Chelsea,0.022757,Saved Off Target,"[92.5, 52.7]",NaN,Left Foot,True,"[118.5, 46.5, 1.6]",NaN,NaN,True,Normal,Open Play,Shot
3272,22,Chelsea,0.008497,Goal,"[101.9, 15.7]",NaN,Right Foot,NaN,"[120.0, 43.1, 0.8]",NaN,NaN,NaN,Normal,Free Kick,Shot
3273,28,Swansea City,0.082658,Saved,"[110.4, 43.0]",True,Head,NaN,"[116.6, 39.4, 0.9]",NaN,NaN,NaN,Normal,Open Play,Shot
3274,28,Swansea City,0.262751,Blocked,"[114.1, 37.9]",NaN,Left Foot,NaN,"[115.6, 38.2]",NaN,NaN,NaN,Normal,Open Play,Shot
3275,28,Swansea City,0.306705,Goal,"[113.1, 36.8]",NaN,Left Foot,NaN,"[120.0, 43.6, 1.3]",NaN,NaN,NaN,Normal,Open Play,Shot
3276,32,Chelsea,0.010350,Saved,"[119.0, 61.3]",NaN,Right Foot,NaN,"[120.0, 43.8, 1.9]",NaN,True,NaN,Normal,Open Play,Shot
